In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import re
from lxml import etree
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
from datetime import datetime

In [19]:
df_listings = pd.read_csv('./listings.csv')
df_reviews = pd.read_csv('./reviews.csv')
df_listings = df_listings.convert_dtypes()
df_reviews = df_reviews.convert_dtypes()
df_listings = df_listings.rename(columns={'id':'listing_id'})


In [20]:
#to drop
cols_to_drop_listings = ['listing_url','scrape_id','last_scraped','source','picture_url','host_url','host_name','host_thumbnail_url',
                'host_picture_url','has_availability','license', 'availability_30','availability_60','availability_90','availability_365',
                'calendar_last_scraped','host_location','bathrooms_text','calendar_updated','host_id']

cols_to_drop_reviews = ['reviewer_name','reviewer_id']

In [21]:
df_listings.drop(columns=cols_to_drop_listings,inplace=True)
df_reviews.drop(columns=cols_to_drop_reviews,inplace=True)
df_reviews.dropna(subset=['comments'],inplace=True)
df_reviews_combined = df_reviews.groupby('listing_id')['comments'].agg(' '.join).reset_index()
row_counts = df_reviews.groupby('listing_id').size().reset_index(name='review_count')
df_reviews = pd.merge(df_reviews_combined, row_counts, on='listing_id').reset_index()
df_combined = pd.merge(df_reviews,df_listings,on='listing_id').reset_index()
df_combined.drop(columns=['level_0','index'],inplace=True)
df_combined['host_response_rate'] = df_combined['host_response_rate'].str.rstrip('%').astype('float') / 100
df_combined.convert_dtypes()

,listing_id,comments,review_count,name,description,neighborhood_overview,host_since,host_about,host_response_time,host_response_rate,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,44077,We enjoyed our stay very much. The room was co...,341,cosy comfortable Irish cottage twin,Our house is a cosy comfortable cottage with ...,I like our neighbourhood as there is no shorta...,2010-08-06,"I am a friendly outgoing Irishwoman ,who love...",within an hour,0.96,...,4.92,4.92,4.67,4.82,f,2,0,2,0,2.11
1,85156,Teresa and family were lovely hosts. They were...,303,Cosy Comfortable Irish Cottage 1 Double Bed,Our Cottage is a charming light filled cottage...,"I love Dundrum and its surrounding areas ,<br ...",2010-08-06,"I am a friendly outgoing Irishwoman ,who love...",within an hour,0.96,...,4.91,4.89,4.65,4.78,f,2,0,2,0,1.9
2,159889,Margaret is very helpful and attentive! She an...,433,Friendly Single Room,Washing can be done at a cost of €5 per load....,Plenty of buses into the city and the area is ...,2011-07-01,Used to host students and now I am looking fo...,within an hour,1.0,...,4.84,4.87,4.64,4.71,f,3,0,3,0,2.9
3,162809,Robbie was very welcoming and we had a very go...,388,5.5 miles south of Dublin City :^),A nice place to relax after the bustle of the ...,"Close to the sea, hill walks and the city - vi...",2011-07-04,I live here on the third floor of my house. T...,within a few hours,0.87,...,4.94,4.97,4.78,4.85,f,2,0,2,0,3.52
4,165828,We had a wonderful time staying in Grainne's a...,76,Pádraig Pearse apt. Kilmainham,"Don't just visit Dublin, experience Dublin in ...","Enjoy a walk along the grand canal, or just ob...",2011-07-07,"I'm Gráinne, originally from Co. Kildare and r...",within a day,0.75,...,4.83,4.76,4.39,4.54,f,4,4,0,0,0.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4733,1174461242357488216,Great property and great host,1,Excellent located 2 bed flat,Excellent located 2 bedroom flat with 2 double...,<NA>,2017-05-09,<NA>,within a few hours,1.0,...,<NA>,<NA>,<NA>,<NA>,f,1,1,0,0,1.0
4734,1174669132225502884,Deeply recommend. Liliane is a wonderful perso...,1,Cosy Room in Duplex/ Raheny,Cosy single room in a 100 meter square Suplex ...,<NA>,2017-02-22,"Hi, Lovely to meet you! I am Liliane. I hav...",within an hour,1.0,...,5.0,5.0,5.0,5.0,f,2,0,2,0,1.0
4735,1175337031194584866,5 star stay!! Great value!! <br/>Sakshi was a ...,1,Room In A Family Home - Dublin (Near Tram Stat...,About Us:<br />We are a friendly family of thr...,<NA>,2021-12-22,<NA>,within an hour,1.0,...,5.0,5.0,5.0,5.0,f,1,0,1,0,1.0
4736,1176027794787210610,"Thank you Blas, everything went perfect house ...",1,Beautiful Rom dublin 8,Enjoy a stylish experience at this centrally-l...,<NA>,2022-03-31,<NA>,within a day,0.64,...,5.0,5.0,5.0,5.0,f,1,0,1,0,1.0


In [24]:
df_combined['host_neighbourhood'].unique()

<StringArray>
[              'Churchtown',                 'Beaumont',
                'Blackrock',               'Kilmainham',
                       <NA>,              'Castleknock',
 'Stoneybatter/Arbour Hill',            'The Liberties',
                 'Mountjoy',                    'Lucan',
 ...
                 'Dundrum ',             'Apartment 18',
                 'dublin 2',       'Raheny - Greendale',
              'Stillorgan ',                'Milltown ',
                'Clonshagh',               'Merrion Rd',
                'Honeypark',             'Sutton South']
Length: 399, dtype: string

In [ ]:
def clean_string(text):
    if(type(text) == str):
        # text = etree.fromstring(text,etree.HTMLParser())
        # text = bs(text,'html.parser').get_text()
        tags = re.compile('<.*?>')
        text = re.sub(tags, '', text)
        rx = re.compile(r"[^A-Za-z0-9\s?!]")
        text = rx.sub(' ', text)
        rx = re.compile(" +")
        text = rx.sub(' ', text).strip()
        if text == 't':
            return 1
        if text == 'f':
            return 0
        try:
            return (datetime.now() - datetime.strptime(text,"%Y %m %d")).days
        except:
            pass
        try:
            numbers = re.findall(r'\d+\.?\d*', text)
            if numbers:
                return float(numbers[0])
        except ValueError:
            pass
        if text == '':
            return None
    return text

In [ ]:
df_corr = df_listings.select_dtypes(['int64','float64'])
corr = df_corr.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()